In [1]:
!pip install plotly nbformat

import sys
sys.path.append('../lib')

import numpy as np
import cv2
import orbslam2

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
if not os.path.exists('../Vocabulary/ORBvoc.txt'):
    !cd Vocabulary && tar -xf ORBvoc.txt.tar.gz

VOC_FILE = '../Vocabulary/ORBvoc.txt'
CONF_FILE = '../../iPhone/iPhone.yaml'
VIDEO_FILE = '../../iPhone/living.mov'

In [3]:
system = orbslam2.System(VOC_FILE, CONF_FILE, orbslam2.System.MONOCULAR)
cap = cv2.VideoCapture(VIDEO_FILE)


ORB-SLAM2 Copyright (C) 2014-2016 Raul Mur-Artal, University of Zaragoza.
This program comes with ABSOLUTELY NO WARRANTY;
This is free software, and you are welcome to redistribute it
under certain conditions. See LICENSE.txt.

Input sensor was set to: Monocular

Loading ORB Vocabulary. This could take a while...
Vocabulary loaded!


Camera Parameters: 
- fx: 1080
- fy: 1080
- cx: 540
- cy: 960
- k1: 0.262383
- k2: -0.953104
- k3: 1.16331
- p1: -0.005358
- p2: 0.002628
- fps: 30
- color order: BGR (ignored if grayscale)

ORB Extractor Parameters: 
- Number of Features: 1000
- Scale Levels: 8
- Scale Factor: 1.2
- Initial Fast Threshold: 20
- Minimum Fast Threshold: 7


In [4]:
import time

t0 = time.time()

while True:
    ret, img = cap.read()
    t = time.time() - t0

    if not ret:
        break

    a = system.track_monocular(img, t)
    #if a is None:
    if system.get_tracking_state() == 3 and t > 10.:
        break

    print(t, a)
    if t > 20.: break

0.11959290504455566 None
0.16540741920471191 None
0.21670961380004883 None
0.2642934322357178 None
0.3121035099029541 None
0.36107873916625977 None
0.4030940532684326 None
0.4407501220703125 None
0.48831772804260254 None
0.5254476070404053 None
0.5626626014709473 None
0.6119167804718018 None
0.6621379852294922 None
0.6998732089996338 None
0.7347075939178467 None
0.7707881927490234 None
0.8089346885681152 None
0.8433065414428711 None
0.8727173805236816 None
0.9047703742980957 None
0.934424638748169 None
0.9752047061920166 None
1.0155112743377686 None
1.0562329292297363 None
1.097620964050293 None
1.1382977962493896 None
1.1757090091705322 None
1.2077834606170654 None
1.2366268634796143 None
1.2749383449554443 None
1.3113651275634766 None
1.3407444953918457 None
1.3702688217163086 None
1.4145021438598633 None
1.4455368518829346 None
1.4767775535583496 None
1.5165514945983887 None
1.5595815181732178 None
1.599456787109375 None
1.6401615142822266 None
1.6797289848327637 None
1.720371484756

In [51]:
import plotly.graph_objects as go

map_points = system.get_tracked_map_points()
positions = np.concatenate([mp.world_pos for mp in map_points if mp is not None], 1)
normals = np.concatenate([mp.normal for mp in map_points if mp is not None], 1)

data=[]
data.append(
    go.Cone(
        x=positions[0],
        y=positions[1],
        z=positions[2],
        u=normals[0],
        v=normals[1],
        w=normals[2],
        #opacity=0.5,
        colorscale=[[0, 'red'], [1, 'red']],
        sizeref=10,
    )
)
"""
data.append(
    go.Cone(
        x=camera_points[0],
        y=camera_points[1],
        z=camera_points[2],
        u=camera_zaxis[0],
        v=camera_zaxis[1],
        w=camera_zaxis[2],
        colorscale=[[0, 'blue'], [1, 'blue']]
    )
)
"""

go.Figure(data=data).show()